

# Distributed arrays

Parallel computing is a mess: too many types of parallelism:
- MPI
- CUDA
- OpenMP
- Threads

Idea: `DistributedArrays` *will* be (currently not quite there yet) the best, and easiest, way to do parallel computing. 

It's an *easy* form of parallelism, when it works.

An array that lives on several processors -- each processor has a part of the array (a "local part"). The array is **partitioned** on the different processors.

Note that the idea of referring to a vector as just one object, `v`, was a big deal in Matlab / Python etc.

Now `v` will refer to an array that is a more complicated object.

A `DistributedArray` will just **look like** (to us) a standard Julia array. The complicated stuff inside is hidden by an **abstraction**.

In [ ]:
using Pkg; Pkg.add("DistributedArrays")

In [ ]:
using Distributed
# Add processes:
addprocs(2)

In [ ]:
using DistributedArrays

In [ ]:
# Use package for "distributed arrays"
@everywhere using DistributedArrays

## Parallelize it (where "it" is embarrassingly parallel)

In [ ]:
procs()

In [ ]:
workers()

In [ ]:
a = [1:1000;]

In [ ]:
# Distribute the data:

D = distribute(a)

In [ ]:
T = typeof(D)

In [ ]:
supertype(T)

To get at the information inside the object:

    `D.<TAB>`

In [ ]:
fieldnames(typeof(D))

Which piece of the `DArray` is stored on each worker:

In [ ]:
D.indices

We want to write the *same* code and have it "just work":

In [ ]:
dD = map(t -> t^2, D)  # dD is the distributed answer

In [ ]:
# apply map to distributed vector (looks identical to non-distributed case)

dD == map(t->t^2, a)  # undistributes the array back onto the master node

In [ ]:
@fetchfrom 3 localpart(dD)  # the result that worker 2 calculated

In [ ]:
@everywhere using Dates

In [ ]:
# Distributed vectors not restricted to numerical types

map(t -> Dates.monthname((t - 1) % 12 + 1), D)

In [ ]:
# A slightly more complicated example of map and reduce

monthString = map(t -> Dates.monthname((t - 1) % 12 + 1) |> s -> s*" is my favorite month.\n", D) |>
    t -> reduce(*, Array(t))
println(monthString)

In [ ]:
# Distributed array comprehension

D55 = @DArray [randn(500,500) for i = 1:32];

In [ ]:
using LinearAlgebra

In [ ]:
# Compute singular values of the distributed vector of matrices: 

@time Dsvd = map(svdvals, D55);

In [ ]:
BLAS.set_num_threads(1)

In [ ]:
d5 = [randn(500,500) for i = 1:32];

In [ ]:
@time map(svdvals, d5);